# *btest* initial steps

step-by-step

In [1]:
import argparse
import csv
import itertools
import logging
import os
import sys
import shutil
import time
import math
import random
from scipy.stats import pearsonr
import numpy as np
import pandas as pd
import math
from scipy.stats import spearmanr
from btest import config
from btest import utils
from btest import blockplot


In [2]:
method = 'spearman'
min_var = 0.0
fdr =0.1

In [3]:
dataX , dataY, featuresX, featuresY  = utils.readData('~/Documents/omicsEye/btest/demo/X_dataset.txt', '~/Documents/omicsEye/btest/demo/Y_dataset.txt', min_var=min_var)

In [4]:
start_time = time.time()
within_X = utils.btest_corr(dataX, featuresX, method=method, fdr=fdr, Type='withinX')
within_Y  = utils.btest_corr(dataY, featuresY, method=method, fdr=fdr, Type='withinY')
dataAll = np.concatenate((dataX, dataY), axis=0)
X_Y = utils.btest_corr(dataAll, featuresX, featuresY, method=method, fdr=fdr, Type='X_Y')
rho_excution_time = time.time() - start_time

In [5]:
rho_excution_time

10.708202123641968

In [6]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
165,X1,Y66,0.021416,0.327972,49,212.013466,0.65493,X_Y
174,X1,Y75,0.033062,0.305052,49,163.658486,0.65493,X_Y
2747,X27,Y74,0.036443,0.296654,50,120.262541,0.65493,X_Y
173,X1,Y74,0.040235,0.294122,49,99.582635,0.65493,X_Y
1526,X15,Y41,0.042550,0.287992,50,84.249800,0.65493,X_Y
...,...,...,...,...,...,...,...,...
7670,X77,Y47,0.999459,-0.000098,50,0.999863,0.65493,X_Y
2271,X22,Y93,0.999729,0.000049,50,1.000032,0.65493,X_Y
5255,X53,Y8,0.999729,0.000049,50,0.999931,0.65493,X_Y
8647,X87,Y34,0.999729,0.000049,50,0.999830,0.65493,X_Y


In [7]:
within_X

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
100,X1,X1,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
394,X4,X4,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
585,X6,X6,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
1134,X12,X12,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
1222,X13,X13,0.000000,1.000000,49,0.000000,9.957214e-16,withinX
...,...,...,...,...,...,...,...,...
3137,X38,X78,0.998917,-0.000197,50,0.999709,9.957214e-16,withinX
3149,X38,X90,0.998917,-0.000197,50,0.999511,9.957214e-16,withinX
3206,X39,X86,0.998917,-0.000197,50,0.999313,9.957214e-16,withinX
4435,X65,X80,0.998917,-0.000197,50,0.999115,9.957214e-16,withinX


In [8]:
within_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold,Type
0,Y0,Y0,0.000000,1.000000,50,0.000000,0.035926,withinY
100,Y1,Y1,0.000000,1.000000,50,0.000000,0.035926,withinY
199,Y2,Y2,0.000000,1.000000,50,0.000000,0.035926,withinY
394,Y4,Y4,0.000000,1.000000,50,0.000000,0.035926,withinY
585,Y6,Y6,0.000000,1.000000,50,0.000000,0.035926,withinY
...,...,...,...,...,...,...,...,...
4002,Y54,Y87,0.993502,-0.001182,50,0.994290,0.035926,withinY
2553,Y29,Y88,0.994587,-0.000984,50,0.995179,0.035926,withinY
487,Y4,Y97,0.995128,-0.000886,50,0.995522,0.035926,withinY
2836,Y33,Y97,0.995128,-0.000886,50,0.995325,0.035926,withinY


In [9]:
X_Y.pval

165     0.021416
174     0.033062
2747    0.036443
173     0.040235
1526    0.042550
          ...   
7670    0.999459
2271    0.999729
5255    0.999729
8647    0.999729
9011    0.999729
Name: pval, Length: 9900, dtype: float64

In [41]:
outputpath = "/Users/rah/Documents/omicsEye/btest/demo/btest_output"
os.makedirs(outputpath, exist_ok=True)
X_Y.to_csv(outputpath + '/X_Y.tsv', sep="\t")
#rho_X = rho.iloc[0:500, 0:500]
#rho_X.to_csv(outputpath + '/simtable_X.tsv', sep="\t")
#rho_X_Y.to_csv(outputpath + '/simtable.tsv', sep="\t")
within_X.to_csv(outputpath + '/within_X.tsv', sep="\t")
within_Y.to_csv(outputpath + '/within_Y.tsv', sep="\t")

In [2]:
#associations = blockplot.load_associations(path = outputpath + '/X_Y.tsv')
associations = blockplot.load_associations('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new/X_Y.tsv')

In [3]:
outputpath

NameError: name 'outputpath' is not defined

In [4]:
associations

[['616', ['GCF00020'], ['cystine'], '0.0', '1.0', '0.0'],
 ['623', ['GCF00020'], ['dihydroorotate'], '0.0', '1.0', '0.0'],
 ['673',
  ['GCF00020'],
  ['hydrochlorothiazide_C18-neg_v2'],
  '0.0',
  '1.0',
  '0.0'],
 ['3031', ['GCF00023'], ['C34:2 DAG'], '0.0', '1.0', '0.0'],
 ['3091', ['GCF00023'], ['C52:7 TAG'], '0.0', '1.0', '0.0'],
 ['3327', ['GCF00023'], ['tartarate'], '0.0', '1.0', '0.0'],
 ['5361', ['GCF00099'], ['2-aminoheptanoic acid'], '0.0', '1.0', '0.0'],
 ['5366', ['GCF00099'], ['2-hydroxyglutarate'], '0.0', '1.0', '0.0'],
 ['5388', ['GCF00099'], ['4-pyridoxate'], '0.0', '1.0', '0.0'],
 ['5414', ['GCF00099'], ['alanylalanine'], '0.0', '1.0', '0.0'],
 ['5420', ['GCF00099'], ['alpha-ketoglutarate'], '0.0', '1.0', '0.0'],
 ['5429', ['GCF00099'], ['arginine_HILIC-pos_v3'], '0.0', '1.0', '0.0'],
 ['5450', ['GCF00099'], ['C14 carnitine'], '0.0', '1.0', '0.0'],
 ['5455', ['GCF00099'], ['C14:1 carnitine'], '0.0', '1.0', '0.0'],
 ['5461', ['GCF00099'], ['C16:0 LPC_C8-pos_v2'], '0.0',

In [5]:
#simtable = blockplot.load_order_table(outputpath + '/simtable.tsv', associations)
simtable = blockplot.load_order_table('/Users/rah/Dropbox/Ali-Docs/Research_docs/Projects/VastBiome/analysis/btest_bgc_mgx_replaceNA_FALSE_cov_25_measure_rpkg_mbx_pair_spearman_new//simtable.tsv', associations)

In [46]:
X_Y

,Feature_1,Feature_2,pval,Correlation,Not_NAs,P_adusted,bh_fdr_threshold
365,X1,Y66,0.021416,0.327972,49,214.155016,0.65493
374,X1,Y75,0.033062,0.305052,49,165.311602,0.65493
5196,X27,Y74,0.036443,0.296654,50,121.477314,0.65493
373,X1,Y74,0.040235,0.294122,49,100.588520,0.65493
3021,X15,Y41,0.042550,0.287992,50,85.100808,0.65493
...,...,...,...,...,...,...,...
12544,X77,Y47,0.999459,-0.000098,50,0.999859,0.65493
4340,X22,Y93,0.999729,0.000049,50,1.000029,0.65493
9277,X53,Y8,0.999729,0.000049,50,0.999929,0.65493
13706,X87,Y34,0.999729,0.000049,50,0.999829,0.65493


In [47]:
blockplot.plot(
    simtable,
    associations,
    cmap="RdBu_r",
    mask=False,
    axlabels=["",""],
    outfile="/Users/rah/Documents/omicsEye/btest/demo/btest_output/blockplot.pdf",
    similarity="Spearman"
)